In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [4]:
configuration = Configuration()
configuration.username = 'exact' #'exact'
configuration.password = 'exact' #'exact'
configuration.host = "http://127.0.0.1:1337" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download Rect annotation

In [5]:
annotation_types_points = {}
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
            
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.name] = annotation_type

annotation_types_points

{'L0': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 21,
  'name': 'L0',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L1': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 22,
  'name': 'L1',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L2': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 23,
  'name': 'L2',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 'L3': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': Fal

In [6]:
annotation_types_ref_rect

{'area_hit_test': True,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 10,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 6,
 'sort_order': 0,
 'vector_type': 1}

In [7]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Multi_IHC", expand="product_set,product_set.annotationtype_set").results[0]


for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id, annotation_type=annotation_types_ref_rect.id, # BoundBox ID
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        annotations.append([image.id, image.name, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "id", "vector", "unique_identifier", "annotation_type"])
annotations.head()

,image_id,image_name,id,vector,unique_identifier,annotation_type
0,334,CRC-A1-1.tif,26231,"{'x1': 11416, 'x2': 149099, 'y1': 2174, 'y2': ...",a67f3a29-b7ce-41b3-8510-478721ad7ec5,10
1,336,CRC-A1-2.tif,26232,"{'x1': 1513, 'x2': 122258, 'y1': 739, 'y2': 14...",81c548d9-dad8-47dc-9792-435488beacb1,10
2,338,CRC-A1-5.tif,26233,"{'x1': 9664, 'x2': 153871, 'y1': 5846, 'y2': 1...",b749b0cb-a7e3-47b0-b70e-666a2434f6dd,10
3,339,CRC-A1-10.tif,26234,"{'x1': 3561, 'x2': 151374, 'y1': 4162, 'y2': 1...",59ce9693-433a-4772-a163-5c1a98928f25,10
4,341,CRC-A1-13.tif,26235,"{'x1': 40, 'x2': 117686, 'y1': 1425, 'y2': 145...",87f879d5-df29-4eed-bf46-44f5e397f3b6,10


In [8]:
num_lines = 5
patch_size = 100

registration_points = []
for id, bounding_box in annotations.iterrows():
    
    box_width = bounding_box.vector["x2"] - bounding_box.vector["x1"]
    box_height = bounding_box.vector["y2"] - bounding_box.vector["y1"]
    
    step_size_x = box_width // num_lines
    step_size_y = box_height // num_lines
        
    point_index = 0
    for x in range(bounding_box.vector["x1"] + step_size_x //2, bounding_box.vector["x2"], step_size_x):
        for y in range(bounding_box.vector["y1"] + step_size_y //2, bounding_box.vector["y2"], step_size_y):
            
            ref_point = annotation_types_points[f"L{point_index}"]
                
            vector = {"x1": x - patch_size, "y1": y - patch_size, 
                          "x2": x + patch_size, "y2": y + patch_size}
                
            point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id]
                
            registration_points.append(point)
                
            point_index += 1
    
registration_points = pd.DataFrame(registration_points, 
                                   columns=["image_id", "vector", "point_index", "point_name", "point_id"])
registration_points

,image_id,vector,point_index,point_name,point_id
0,334,"{'x1': 25084, 'y1': 16467, 'x2': 25284, 'y2': ...",0,L0,21
1,334,"{'x1': 25084, 'y1': 45253, 'x2': 25284, 'y2': ...",1,L1,22
2,334,"{'x1': 25084, 'y1': 74039, 'x2': 25284, 'y2': ...",2,L2,23
3,334,"{'x1': 25084, 'y1': 102825, 'x2': 25284, 'y2':...",3,L3,24
4,334,"{'x1': 25084, 'y1': 131611, 'x2': 25284, 'y2':...",4,L4,25
...,...,...,...,...,...
120,341,"{'x1': 105820, 'y1': 15738, 'x2': 106020, 'y2'...",20,L20,41
121,341,"{'x1': 105820, 'y1': 44564, 'x2': 106020, 'y2'...",21,L21,42
122,341,"{'x1': 105820, 'y1': 73390, 'x2': 106020, 'y2'...",22,L22,43
123,341,"{'x1': 105820, 'y1': 102216, 'x2': 106020, 'y2...",23,L23,44


## Upload ref Points to EXACT

In [9]:
annotations_to_upload = []

for id, bounding_box in tqdm(registration_points.iterrows()):
    
    
    exact_annos =  annotations_api.list_annotations(image=bounding_box.image_id, annotation_type=bounding_box.point_id).results
    
    if len(exact_annos) == 0:
        anno = Annotation(annotation_type=bounding_box.point_id, 
                          vector=bounding_box.vector, 
                          image=bounding_box.image_id)

        annotations_to_upload.append(anno)

    else:
        prinf(f"ImageId: {bounding_box.image_id}, AnnotationType: {bounding_box.point_id} combination allready exists!")

125it [00:44,  2.79it/s]


In [10]:
annotations_to_upload[:3]

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 334,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 25084, 'x2': 25284, 'y1': 16467, 'y2': 16667},
  'verified_by_user': None},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 334,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 25084, 'x2': 25284, 'y1': 45253, 'y2': 45453},
  'verified_by_user': None},
 {'annotation_type': 23,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,


## Check that no error occurred and afterwards upload annotations

In [11]:
annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26236,
  'image': 334,
  'last_edit_time': datetime.datetime(2021, 2, 10, 9, 1, 40, 821846),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 10, 9, 1, 40, 821823),
  'unique_identifier': 'd68e7b9a-d3ae-43ea-9c7c-4dcac427c588',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 25084, 'x2': 25284, 'y1': 16467, 'y2': 16667},
  'verified_by_user': 'False'},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26237,
  'image': 334,
  'last_edit_time': datetime.datetime(2021, 2, 10, 9, 1, 40, 821908),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 10, 9, 1, 40, 821894),
  'unique_identifier': '0ca22054-a283-4010-bde9-ecb8482c1bd3',
  'uploaded_media_files': [],
  'user': 2,
